# Notes
This code is designed for examine whether the coding property of the prior map is `heterogeneous` along the main track.

This involves three aspects:
1. Cross-session stability
2. Spatial Information
3. Decoding Error.

In [1]:
from mylib.statistic_test import *
from mylib.decoder.NaiveBayesianDecoder import NaiveBayesDecoder

code_id = "0862 - Test Coding Property Heterogeneity"
loc = join(figpath, "Dsp", code_id)
mkdir(loc)
pass
a = np.array([[0, 1], [2, 3]])
print(np.where(a >= 1))

        E:\Data\FinalResults\Dsp\0862 - Test Coding Property Heterogeneity is already existed!
(array([0, 1, 1], dtype=int64), array([1, 0, 1], dtype=int64))


In [ ]:
decode_dir = join(loc, "Decoding Results")

def gnb_decoder(trace):
    """
    Decode the trace using the GNB decoder.
    """
    beg, end = LapSplit(trace, trace['paradigm'])
    beg_t, end_t = trace['lap beg time'], trace['lap end time']
    routes = classify_lap(spike_nodes_transform(trace['correct_nodes'], 12), beg)
    
    raw_traces = trace['RawTraces']
    ms_time = trace['ms_time']
    spike_nodes = trace['spike_nodes_original']
    
    idx = np.concatenate([
        np.where(
            (ms_time >= beg_t[i]) & (ms_time <= end_t[i]) & (np.isnan(spike_nodes) == False)
        )[0] for i in np.where(routes == 0)[0]
    ])
    
    converted_spikes = np.where(raw_traces - np.std(raw_traces, axis=1)[:, np.newaxis]*3 >= 0, 1, 0)
    Spikes = converted_spikes[:, idx]
    spike_nodes = spike_nodes[idx].astype(np.int64)
    ms_time = ms_time[idx]
    
    one_fraction = int(idx.shape[0] / 5)
    start_idx = np.arange(0, idx.shape[0]+1, one_fraction)
    
    mean_loss = np.zeros((5, 4))
    D = GetDMatrices(1, 48)
    # Validate for 5 times
    for i in range(5):
        print(f"Iter {i+1}")
        test_idx = np.arange(start_idx[i], start_idx[i+1])
        train_idx = np.setdiff1d(np.arange(spike_nodes.shape[0]), test_idx)
        
        model = NaiveBayesDecoder(maze_type=1, res=48, smooth_matrix=trace['Ms'])
        model.fit(Spikes[:, train_idx], spike_nodes[train_idx])
        y_pred = model.predict(Spikes_test=Spikes[:, test_idx], MazeID_test=spike_nodes[test_idx])
        y_test = spike_nodes[test_idx]
        
        loss = D[(y_pred-1, y_test-1)]
        converted_y_test = S2F[y_test-1]
        for j, bin_range in enumerate([CP_DSP[0][:27], CP_DSP[0][27:54], CP_DSP[0][54:81], CP_DSP[3]]):
            in_range_idx = np.where(np.isin(converted_y_test, bin_range))[0]
            mean_loss[i, j] = np.nanmedian(loss[in_range_idx])
            
        print(f"  Mean Losses Are:      {mean_loss[i, 0], mean_loss[i, 1], mean_loss[i, 2], mean_loss[i, 3]}")
            
    return mean_loss
        

for j, i in enumerate(np.where(f2['MiceID'] == 10227)[0]):
    if j == 0:
        continue
    
    print(f2['MiceID'][i], f"S{j+1} ---------------------------")
    with open(f2['Trace File'][i], 'rb') as handle:
        trace = pickle.load(handle)

    mean_loss = gnb_decoder(trace)
    
    with open(join(decode_dir, f"Loss_{f2['MiceID'][i]}_{i}.pkl"), 'wb') as f:
        pickle.dump(mean_loss, f)
        
    print()


10227 S1 ---------------------------
Iter 1
    Generate D matrix
    D matrix successfully generated!
    Generating tuning curve
    Tuning curve successfully generated!
    Generating P matirx...


100%|██████████| 5106/5106 [05:30<00:00, 15.46it/s]


  Mean Losses Are:      (10.233345472033855, 7.211102550927978, 8.246211251235321, 6.324555320336759)
Iter 2
    Generate D matrix
    D matrix successfully generated!
    Generating tuning curve
    Tuning curve successfully generated!
    Generating P matirx...


100%|██████████| 5106/5106 [06:26<00:00, 13.20it/s]


  Mean Losses Are:      (10.929971408438085, 8.246211251235321, 10.901046542878234, 5.656854249492381)
Iter 3
    Generate D matrix
    D matrix successfully generated!
    Generating tuning curve
    Tuning curve successfully generated!
    Generating P matirx...


...

100%|██████████| 5475/5475 [04:48<00:00, 19.00it/s]


  Mean Losses Are:      (12.0, 10.0, 10.0, 8.246211251235321)
Iter 4
    Generate D matrix
    D matrix successfully generated!
    Generating tuning curve
    Tuning curve successfully generated!
    Generating P matirx...


100%|██████████| 5475/5475 [03:52<00:00, 23.55it/s]


  Mean Losses Are:      (12.902019706710776, 14.142135623730951, 13.416407864998739, 7.211102550927978)
Iter 5


IndexError: index 5 is out of bounds for axis 0 with size 5